内存分配器：
- 以某个阈值为界，分⼤小对象
- 大对象直接分配内存，只小对象才使⽤用专⽤用内存分配器器
- 在通⽤用内存分配基础上，以缓存⽅方式实现小对象快速分配

python3.6中，大小对象以512字节为界，Arena为512KB，Pool为4KB。

|Request in bytes |Size of allocated block |Size class idx|
|-----------------|-------------------------|----------------------| 
|1-8 |8 |0|
|9-16 |16 |1|
|17-24 |24 |2|
|25-32 |32 |3|
|... |... |...|
|497-504 |504 |62|
|505-512 |512 |63|

小对象按8字节对齐，分成64个不同的类别。

- 向系统申请⼤块 arena 内存
- 分成多个 pool 块(⼀级重⽤用单元)，为⼀种类别提供内存
- 按类别，分成多个 block 块(⼆级)。可存储相同类别的⼩对象

分配流程
每个大块内存都用专门的对象进行管理，以存储相关状态。  
Objects/obmalloc.c
```
static struct arena_object*
new_arena(void)
{
    struct arena_object* arenaobj;
    uint excess;        /* number of bytes above pool alignment */
    void *address;
    static int debug_stats = -1;
    
    // 没有可用的管理对象
    if (unused_arena_objects == NULL) {
        uint i;
        uint numarenas;
        size_t nbytes;

        // 确定要分配的管理对象数量，并分配内存
        numarenas = maxarenas ? maxarenas << 1 : INITIAL_ARENA_OBJECTS;
        nbytes = numarenas * sizeof(*arenas);
        arenaobj = (struct arena_object *)PyMem_RawRealloc(arenas, nbytes);
        
        // 使用链表保存管理对象
        arenas = arenaobj;
        /* Put the new arenas on the unused_arena_objects list. */
        for (i = maxarenas; i < numarenas; ++i) {
            arenas[i].address = 0;              /* mark as unassociated */
            arenas[i].nextarena = i < numarenas - 1 ?
                                   &arenas[i+1] : NULL;
        }
        /* Update globals. */
        unused_arena_objects = &arenas[maxarenas];
        maxarenas = numarenas;
    }

    // 提取一个管理对象
    arenaobj = unused_arena_objects;
    unused_arena_objects = arenaobj->nextarena;
    assert(arenaobj->address == 0);
    // 为其分配大块内存
    address = _PyObject_Arena.alloc(_PyObject_Arena.ctx, ARENA_SIZE);  // 256KB
    if (address == NULL) {
        /* The allocation failed: return NULL after putting the
         * arenaobj back.
         */
        arenaobj->nextarena = unused_arena_objects;
        unused_arena_objects = arenaobj;
        return NULL;
    }
    arenaobj->address = (uintptr_t)address;

    ++narenas_currently_allocated;
    ++ntimes_arena_allocated;
    if (narenas_currently_allocated > narenas_highwater)
        narenas_highwater = narenas_currently_allocated;
    // 初始化其pool的相关属性
    arenaobj->freepools = NULL;
    arenaobj->pool_address = (block*)arenaobj->address;
    arenaobj->nfreepools = ARENA_SIZE / POOL_SIZE;  // 4KB
    assert(POOL_SIZE * arenaobj->nfreepools == ARENA_SIZE);
    excess = (uint)(arenaobj->address & POOL_SIZE_MASK);
    if (excess != 0) {
        --arenaobj->nfreepools;
        arenaobj->pool_address += POOL_SIZE - excess;
    }
    arenaobj->ntotalpools = arenaobj->nfreepools;

    return arenaobj;
}

```
arenaobj用来管理大块内存，会一次性分配多个备用，以减少内存碎片。但大块内存申请是按请求单词分配的，与管理对象无关。

PyObject：
Include/objimpl.h
```
 - PyObject_New(type, typeobj) allocates memory for a new object of the given
   type, and initializes part of it.  'type' must be the C structure type used
   to represent the object, and 'typeobj' the address of the corresponding
   type object.  Reference count and type pointer are filled in; the rest of
   the bytes of the object are *undefined*!  The resulting expression type is
   'type *'.  The size of the object is determined by the tp_basicsize field
   of the type object.

 - PyObject_NewVar(type, typeobj, n) is similar but allocates a variable-size
   object with room for n items.  In addition to the refcount and type pointer
   fields, this also fills in the ob_size field.
```
Objects/object.c
```
PyObject *
_PyObject_New(PyTypeObject *tp)
{
    PyObject *op;
    op = (PyObject *) PyObject_MALLOC(_PyObject_SIZE(tp));
    return PyObject_INIT(op, tp);
}
```
Objects/obmalloc.c
```
static void *
_PyObject_Malloc(void *ctx, size_t nbytes)
{
    void* ptr;
    if (pymalloc_alloc(ctx, &ptr, nbytes)) {
        _Py_AllocatedBlocks++;
        return ptr;
    }

    ptr = PyMem_RawMalloc(nbytes);
    if (ptr != NULL) {
        _Py_AllocatedBlocks++;
    }
    return ptr;
}
```

```
/* pymalloc allocator

   The basic blocks are ordered by decreasing execution frequency,
   which minimizes the number of jumps in the most common cases,
   improves branching prediction and instruction scheduling (small
   block allocations typically result in a couple of instructions).
   Unless the optimizer reorders everything, being too smart...

   Return 1 if pymalloc allocated memory and wrote the pointer into *ptr_p.

   Return 0 if pymalloc failed to allocate the memory block: on bigger
   requests, on error in the code below (as a last chance to serve the request)
   or when the max memory limit has been reached. */
static int
pymalloc_alloc(void *ctx, void **ptr_p, size_t nbytes)
{
    block *bp;
    poolp pool;
    poolp next;
    uint size;

    size = (uint)(nbytes - 1) >> ALIGNMENT_SHIFT;
    // 1. 从管理链表（usedpools）提取正在使用的pool
    pool = usedpools[size + size];
    if (pool != pool->nextpool) {
        /*
         * There is a used pool for this size class.
         * Pick up the head block of its free list.
         */
        ++pool->ref.count;
        // 从回收的复用链表（freeblock）中提取block
        bp = pool->freeblock;
        assert(bp != NULL);
        if ((pool->freeblock = *(block **)bp) != NULL) {
            goto success;
        }

        // 从未分配的内存中切分block
        if (pool->nextoffset <= pool->maxnextoffset) {
            /* There is room for another block. */
            pool->freeblock = (block*)pool +
                              pool->nextoffset;
            pool->nextoffset += INDEX2SIZE(size);
            *(block **)(pool->freeblock) = NULL;
            goto success;
        }

        /* Pool is full, unlink from used pools. */
        next = pool->nextpool;
        pool = pool->prevpool;
        next->prevpool = pool;
        pool->nextpool = next;
        goto success;
    }
    // 2. 从arean获取新的pool
    if (usable_arenas == NULL) {
        /* No arena has a free pool:  allocate a new arena. */
        // 申请新的arena
        usable_arenas = new_arena();
        if (usable_arenas == NULL) {
            goto failed;
        }
        usable_arenas->nextarena =
            usable_arenas->prevarena = NULL;
    }
    // 从回收的复用链表（freepools）中提取pool
    pool = usable_arenas->freepools;
    if (pool != NULL) {
        /* Unlink from cached pools. */
        // 调整链表和可用计数
        usable_arenas->freepools = pool->nextpool;

    init_pool:
        /* Frontlink to used pools. */
        // 添加到管理链表(usedpools)
        next = usedpools[size + size]; /* == prev */
        pool->nextpool = next;
        pool->prevpool = next;
        next->nextpool = pool;
        next->prevpool = pool;
        pool->ref.count = 1;
        // 初始化block属性
        pool->szidx = size;
        size = INDEX2SIZE(size);
        bp = (block *)pool + POOL_OVERHEAD;
        pool->nextoffset = POOL_OVERHEAD + (size << 1);
        pool->maxnextoffset = POOL_SIZE - size;
        pool->freeblock = bp + size;
        *(block **)(pool->freeblock) = NULL;
        goto success;
    }

    // 从arena切分新的pool
    pool = (poolp)usable_arenas->pool_address;
    pool->arenaindex = (uint)(usable_arenas - arenas);
    pool->szidx = DUMMY_SIZE_IDX;
    // 调整位置和数量信息
    usable_arenas->pool_address += POOL_SIZE;
    --usable_arenas->nfreepools;
    // 将pool添加至管理链表，并初始化block属性
    goto init_pool;

success:
    assert(bp != NULL);
    *ptr_p = (void *)bp;
    return 1;

failed:
    return 0;
}

```

释放操作：
只是简单的将Block添加到回收链表，但会检查Pool和Arena是否可被回收，并尝试释放其占用的内存。  
Objects/obmalloc.c
```
static void
_PyObject_Free(void *ctx, void *p)
{
    /* PyObject_Free(NULL) has no effect */
    if (p == NULL) {
        return;
    }

    _Py_AllocatedBlocks--;
    if (!pymalloc_free(ctx, p)) {
        /* pymalloc didn't allocate this address */
        // 释放大对象内存
        PyMem_RawFree(p);
    }
}
```

```
/* Free a memory block allocated by pymalloc_alloc().
   Return 1 if it was freed.
   Return 0 if the block was not allocated by pymalloc_alloc(). */
static int
pymalloc_free(void *ctx, void *p)
{
    poolp pool;
    block *lastfree;
    poolp next, prev;
    uint size;
    // 计算所属的pool
    pool = POOL_ADDR(p);
    if (!address_in_range(p, pool)) {
        return 0;
    }
    // 将block保存到回收的复用链表（freeblock）
    *(block **)p = lastfree = pool->freeblock;
    pool->freeblock = (block *)p;

    struct arena_object* ao;
    uint nf;  /* ao->nfreepools */

    // 检查pool被引用的block数量
    if (--pool->ref.count != 0) {
        /* pool isn't empty:  leave it in usedpools */
        goto success;
    }
    /* Pool is now empty:  unlink from usedpools, and
     * link to the front of freepools.  This ensures that
     * previously freed pools will be allocated later
     * (being not referenced, they are perhaps paged out).
     */
    next = pool->nextpool;
    prev = pool->prevpool;
    next->prevpool = prev;
    prev->nextpool = next;

    /* Link the pool to freepools.  This is a singly-linked
     * list, and pool->prevpool isn't used there.
     */
     // 回收自由pool，归还给arena复用链表
    ao = &arenas[pool->arenaindex];
    pool->nextpool = ao->freepools;
    ao->freepools = pool;
    nf = ++ao->nfreepools;

    // 如果arena收回全部的pool，则释放其内存
    if (nf == ao->ntotalpools) {
        
        ao->nextarena = unused_arena_objects;
        unused_arena_objects = ao;

        /* Free the entire arena. */
        _PyObject_Arena.free(_PyObject_Arena.ctx,
                             (void *)ao->address, ARENA_SIZE);
        ao->address = 0;                        /* mark unassociated */
        --narenas_currently_allocated;

        goto success;
    }

    if (nf == 1) {
        /* Case 2.  Put ao at the head of
         * usable_arenas.  Note that because
         * ao->nfreepools was 0 before, ao isn't
         * currently on the usable_arenas list.
         */
        ao->nextarena = usable_arenas;
        ao->prevarena = NULL;
        if (usable_arenas)
            usable_arenas->prevarena = ao;
        usable_arenas = ao;
        assert(usable_arenas->address != 0);

        goto success;
    }

    ao->nextarena->prevarena = ao->prevarena;

    /* Locate the new insertion point by iterating over
     * the list, using our nextarena pointer.
     */
    while (ao->nextarena != NULL && nf > ao->nextarena->nfreepools) {
        ao->prevarena = ao->nextarena;
        ao->nextarena = ao->nextarena->nextarena;
    }


    goto success;

success:
    return 1;
}
```